In [1]:
# Cell 1: Import các thư viện cần thiết (đơn giản)

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print("✅ Đã import thành công!")

✅ Đã import thành công!


In [2]:
# Cell 2: Kết nối Google Drive và đọc dữ liệu

# Kết nối Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Đã kết nối Google Drive thành công!")


Mounted at /content/drive
✅ Đã kết nối Google Drive thành công!


In [3]:
# Đọc file dữ liệu
file_path = '/content/drive/MyDrive/Ebooks/content_base_filtering.csv'
df = pd.read_csv(file_path)

# Kiểm tra dữ liệu cơ bản
print(f"📊 Kích thước dữ liệu: {df.shape}")
print(f"📚 Số lượng sách: {len(df)} cuốn")
print("\n🔍 5 dòng đầu tiên:")
print(df.head())

print("\n📋 Thông tin các cột:")
print(df.info())

📊 Kích thước dữ liệu: (17497, 8)
📚 Số lượng sách: 17497 cuốn

🔍 5 dòng đầu tiên:
                    user_id                   book_id  rating  \
0  688f391ca18786574ae52478  6887514a9e6faeb381cc3a43       4   
1  688f391ca18786574ae52478  68859b92dbd88552ae1fe3f5       4   
2  688f391ca18786574ae52478  68886c8a4059e42f0320e856       4   
3  688f391ca18786574ae52478  6884b3db381cc6a0ba8b7e90       1   
4  688f391ca18786574ae52478  68886a694059e42f0320e7fd       4   

               user_faculty                             book_title  \
0  Khoa Công nghệ Thông tin  Sáng tạo bừng cháy sức mạnh bên trong   
1  Khoa Công nghệ Thông tin              Phong cách học tiếng Việt   
2  Khoa Công nghệ Thông tin                        Trí tuệ Do Thái   
3  Khoa Công nghệ Thông tin           Khai thác dữ liệu với Python   
4  Khoa Công nghệ Thông tin            Nền giáo dục của người giàu   

                                    book_description        book_category  \
0  Sáng Tạo – Bừng Cháy Sức Mạ

In [6]:
# Cell 3: Chuẩn bị dữ liệu cho Content-Based Filtering (đã sửa)

print("🔧 CHUẨN BỊ DỮ LIỆU")
print("="*50)

# 1. Drop cột rating (đơn giản hóa)
df_clean = df.drop(columns=['rating'])
print("✅ Đã loại bỏ cột rating")

# 2. Xóa tất cả dòng của user cụ thể
user_to_remove = "688c54f1b80dd6dd7a0afed0"
before_count = len(df_clean)
df_clean = df_clean[df_clean['user_id'] != user_to_remove]
after_count = len(df_clean)
removed_count = before_count - after_count

print(f"🗑️ Đã xóa {removed_count} dòng của user: {user_to_remove}")
print(f"📊 Dữ liệu còn lại: {after_count} dòng")

# 3. Tạo dataset sách duy nhất (loại bỏ duplicate)
books_df = df_clean.drop_duplicates(subset=['book_id']).reset_index(drop=True)
print(f"\n📚 Dataset sách duy nhất: {len(books_df)} cuốn")

# 4. Hiển thị mẫu dữ liệu đã clean
print(f"\n🔍 Cấu trúc dữ liệu sau khi clean:")
print(books_df.columns.tolist())
print(f"\n📋 Mẫu dữ liệu:")
print(books_df[['book_title', 'book_category', 'book_document_type']].head())

# 5. Kiểm tra user còn lại
remaining_users = df_clean['user_id'].nunique()
print(f"\n👥 Số user còn lại: {remaining_users}")

🔧 CHUẨN BỊ DỮ LIỆU
✅ Đã loại bỏ cột rating
🗑️ Đã xóa 1 dòng của user: 688c54f1b80dd6dd7a0afed0
📊 Dữ liệu còn lại: 17496 dòng

📚 Dataset sách duy nhất: 120 cuốn

🔍 Cấu trúc dữ liệu sau khi clean:
['user_id', 'book_id', 'user_faculty', 'book_title', 'book_description', 'book_category', 'book_document_type']

📋 Mẫu dữ liệu:
                              book_title        book_category  \
0  Sáng tạo bừng cháy sức mạnh bên trong            Triết học   
1              Phong cách học tiếng Việt             Giáo dục   
2                        Trí tuệ Do Thái              Kinh tế   
3           Khai thác dữ liệu với Python  Công nghệ thông tin   
4            Nền giáo dục của người giàu              Kinh tế   

  book_document_type  
0              other  
1           textbook  
2              other  
3           textbook  
4              other  

👥 Số user còn lại: 940


In [7]:
# Cell 4: Tiền xử lý text tiếng Việt
import re
import string

def clean_vietnamese_text(text):
    """
    Hàm làm sạch text tiếng Việt
    - Giữ nguyên dấu tiếng Việt
    - Loại bỏ kí tự đặc biệt
    - Chuẩn hóa khoảng trắng
    """
    if pd.isna(text) or text == "":
        return ""

    # Chuyển về chữ thường
    text = text.lower()

    # Loại bỏ các kí tự đặc biệt (giữ lại chữ, số, space)
    # Giữ lại: a-z, A-Z, 0-9, space, và các kí tự tiếng Việt có dấu
    text = re.sub(r'[^\w\s]', ' ', text)

    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text)

    # Loại bỏ khoảng trắng ở đầu và cuối
    text = text.strip()

    return text

print("🧹 TIỀN XỬ LÝ TEXT TIẾNG VIỆT")
print("="*50)

# Test hàm clean với vài ví dụ
test_texts = [
    "Khai thác dữ liệu với Python - Machine Learning",
    "Triết học & Đạo đức kinh doanh (2023)",
    "Công nghệ thông tin!!! Cơ sở dữ liệu???"
]

print("🔍 TEST HÀM CLEAN TEXT:")
for original in test_texts:
    cleaned = clean_vietnamese_text(original)
    print(f"Gốc:     {original}")
    print(f"Cleaned: {cleaned}")
    print("-" * 40)

# Áp dụng cho dataset sách
print("📚 ÁP DỤNG CHO DATASET SÁCH:")
print("Đang xử lý title và description...")

# Clean title
books_df['title_clean'] = books_df['book_title'].apply(clean_vietnamese_text)

# Clean description
books_df['description_clean'] = books_df['book_description'].apply(clean_vietnamese_text)

print("✅ Hoàn thành tiền xử lý!")
print(f"📊 Số sách đã xử lý: {len(books_df)}")

# Hiển thị kết quả mẫu
print("\n📋 VÍ DỤ KẾT QUẢ:")
sample_df = books_df[['book_title', 'title_clean', 'book_description', 'description_clean']].head(3)
for idx, row in sample_df.iterrows():
    print(f"\n📖 Sách {idx+1}:")
    print(f"Title gốc: {row['book_title']}")
    print(f"Title clean: {row['title_clean']}")
    print(f"Description gốc: {row['book_description'][:100]}...")
    print(f"Description clean: {row['description_clean'][:100]}...")

🧹 TIỀN XỬ LÝ TEXT TIẾNG VIỆT
🔍 TEST HÀM CLEAN TEXT:
Gốc:     Khai thác dữ liệu với Python - Machine Learning
Cleaned: khai thác dữ liệu với python machine learning
----------------------------------------
Gốc:     Triết học & Đạo đức kinh doanh (2023)
Cleaned: triết học đạo đức kinh doanh 2023
----------------------------------------
Gốc:     Công nghệ thông tin!!! Cơ sở dữ liệu???
Cleaned: công nghệ thông tin cơ sở dữ liệu
----------------------------------------
📚 ÁP DỤNG CHO DATASET SÁCH:
Đang xử lý title và description...
✅ Hoàn thành tiền xử lý!
📊 Số sách đã xử lý: 120

📋 VÍ DỤ KẾT QUẢ:

📖 Sách 1:
Title gốc: Sáng tạo bừng cháy sức mạnh bên trong
Title clean: sáng tạo bừng cháy sức mạnh bên trong
Description gốc: Sáng Tạo – Bừng Cháy Sức Mạnh Bên Trong là một trong những tác phẩm giàu cảm hứng nhất của Osho, bậc...
Description clean: sáng tạo bừng cháy sức mạnh bên trong là một trong những tác phẩm giàu cảm hứng nhất của osho bậc th...

📖 Sách 2:
Title gốc: Phong cách học tiếng Việ

In [8]:
# Cell 5: Tạo TF-IDF vectors cho Title và Description
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

print("🔢 TẠO TF-IDF VECTORS")
print("="*50)

# Danh sách stopwords tiếng Việt cơ bản
vietnamese_stopwords = [
    'của', 'và', 'với', 'trong', 'từ', 'cho', 'về', 'là', 'có', 'được',
    'các', 'một', 'để', 'như', 'những', 'này', 'đó', 'khi', 'nếu', 'theo',
    'hoặc', 'nhưng', 'mà', 'thì', 'sẽ', 'đã', 'hay', 'còn', 'chỉ', 'cũng'
]

print(f"📝 Sử dụng {len(vietnamese_stopwords)} stopwords tiếng Việt")

# 1. TF-IDF cho Title
print("\n📖 TẠO TF-IDF CHO TITLE:")
title_vectorizer = TfidfVectorizer(
    max_features=1000,        # Giới hạn 1000 từ quan trọng nhất
    stop_words=vietnamese_stopwords,
    ngram_range=(1, 2),       # Unigram và bigram
    min_df=2,                 # Từ phải xuất hiện ít nhất 2 lần
    max_df=0.8               # Loại bỏ từ xuất hiện quá 80% documents
)

title_tfidf_matrix = title_vectorizer.fit_transform(books_df['title_clean'])
print(f"✅ Title TF-IDF shape: {title_tfidf_matrix.shape}")
print(f"📊 Số từ trong vocabulary: {len(title_vectorizer.vocabulary_)}")

# 2. TF-IDF cho Description
print("\n📄 TẠO TF-IDF CHO DESCRIPTION:")
desc_vectorizer = TfidfVectorizer(
    max_features=2000,        # Description dài hơn nên cho nhiều features hơn
    stop_words=vietnamese_stopwords,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8
)

desc_tfidf_matrix = desc_vectorizer.fit_transform(books_df['description_clean'])
print(f"✅ Description TF-IDF shape: {desc_tfidf_matrix.shape}")
print(f"📊 Số từ trong vocabulary: {len(desc_vectorizer.vocabulary_)}")

# 3. Hiển thị top keywords
print("\n🔍 TOP 10 TỪ KHÓA QUAN TRỌNG:")
print("\n📖 Title keywords:")
title_feature_names = title_vectorizer.get_feature_names_out()
title_scores = np.array(title_tfidf_matrix.sum(axis=0)).flatten()
title_top_indices = title_scores.argsort()[-10:][::-1]
for i, idx in enumerate(title_top_indices):
    print(f"  {i+1}. {title_feature_names[idx]} (score: {title_scores[idx]:.2f})")

print("\n📄 Description keywords:")
desc_feature_names = desc_vectorizer.get_feature_names_out()
desc_scores = np.array(desc_tfidf_matrix.sum(axis=0)).flatten()
desc_top_indices = desc_scores.argsort()[-10:][::-1]
for i, idx in enumerate(desc_top_indices):
    print(f"  {i+1}. {desc_feature_names[idx]} (score: {desc_scores[idx]:.2f})")

# 4. Lưu kết quả để sử dụng ở cell tiếp theo
print(f"\n💾 ĐÃ TẠO XONG TF-IDF VECTORS!")
print(f"📏 Tổng dimensions: Title({title_tfidf_matrix.shape[1]}) + Description({desc_tfidf_matrix.shape[1]})")

🔢 TẠO TF-IDF VECTORS
📝 Sử dụng 30 stopwords tiếng Việt

📖 TẠO TF-IDF CHO TITLE:
✅ Title TF-IDF shape: (120, 202)
📊 Số từ trong vocabulary: 202

📄 TẠO TF-IDF CHO DESCRIPTION:
✅ Description TF-IDF shape: (120, 2000)
📊 Số từ trong vocabulary: 2000

🔍 TOP 10 TỪ KHÓA QUAN TRỌNG:

📖 Title keywords:
  1. học (score: 5.92)
  2. trình (score: 3.69)
  3. lý (score: 3.62)
  4. thuật (score: 3.43)
  5. lập (score: 3.34)
  6. lập trình (score: 3.34)
  7. công (score: 3.03)
  8. kỹ (score: 2.93)
  9. kỹ thuật (score: 2.93)
  10. sản (score: 2.87)

📄 Description keywords:
  1. không (score: 5.38)
  2. giáo (score: 5.30)
  3. chương (score: 5.18)
  4. giáo trình (score: 4.86)
  5. sản (score: 4.41)
  6. công (score: 3.99)
  7. sinh (score: 3.88)
  8. thực (score: 3.75)
  9. lý (score: 3.68)
  10. liệu (score: 3.58)

💾 ĐÃ TẠO XONG TF-IDF VECTORS!
📏 Tổng dimensions: Title(202) + Description(2000)


In [9]:
# Cell 6: One-Hot Encoding cho Category và Document Type
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

print("🏷️ ONE-HOT ENCODING CHO CATEGORICAL FEATURES")
print("="*50)

# 1. Phân tích categories và document types
print("📊 PHÂN TÍCH DỮ LIỆU CATEGORICAL:")
print(f"\n📚 Các thể loại sách (Categories):")
categories = books_df['book_category'].value_counts()
print(categories)

print(f"\n📄 Các loại tài liệu (Document Types):")
doc_types = books_df['book_document_type'].value_counts()
print(doc_types)

# 2. One-Hot Encoding cho Category
print(f"\n🔄 THỰC HIỆN ONE-HOT ENCODING:")
category_encoder = OneHotEncoder(sparse_output=False, dtype=int)
category_encoded = category_encoder.fit_transform(books_df[['book_category']])

print(f"✅ Category encoding shape: {category_encoded.shape}")
print(f"📋 Category labels: {list(category_encoder.categories_[0])}")

# 3. One-Hot Encoding cho Document Type
doc_type_encoder = OneHotEncoder(sparse_output=False, dtype=int)
doc_type_encoded = doc_type_encoder.fit_transform(books_df[['book_document_type']])

print(f"✅ Document type encoding shape: {doc_type_encoded.shape}")
print(f"📋 Document type labels: {list(doc_type_encoder.categories_[0])}")

# 4. Tạo DataFrame để dễ hiểu
print(f"\n📋 VÍ DỤ ENCODING RESULTS:")

# Category DataFrame
category_df = pd.DataFrame(
    category_encoded,
    columns=[f"category_{cat}" for cat in category_encoder.categories_[0]]
)
print(f"\n🏷️ Category One-Hot (5 sách đầu):")
print(category_df.head())

# Document Type DataFrame
doc_type_df = pd.DataFrame(
    doc_type_encoded,
    columns=[f"doctype_{dtype}" for dtype in doc_type_encoder.categories_[0]]
)
print(f"\n📄 Document Type One-Hot (5 sách đầu):")
print(doc_type_df.head())

# 5. Kết hợp với thông tin sách để kiểm tra
print(f"\n🔍 KIỂM TRA KẾT QUẢ ENCODING:")
check_df = pd.concat([
    books_df[['book_title', 'book_category', 'book_document_type']].head(),
    category_df.head(),
    doc_type_df.head()
], axis=1)

for idx in range(3):
    print(f"\n📖 Sách {idx+1}: {check_df.iloc[idx]['book_title']}")
    print(f"   Category: {check_df.iloc[idx]['book_category']}")
    print(f"   Doc Type: {check_df.iloc[idx]['book_document_type']}")

    # Hiển thị which categories = 1
    cat_cols = [col for col in check_df.columns if col.startswith('category_')]
    active_cats = [col.replace('category_', '') for col in cat_cols if check_df.iloc[idx][col] == 1]
    print(f"   Category encoding: {active_cats}")

    # Hiển thị which doc types = 1
    doc_cols = [col for col in check_df.columns if col.startswith('doctype_')]
    active_docs = [col.replace('doctype_', '') for col in doc_cols if check_df.iloc[idx][col] == 1]
    print(f"   DocType encoding: {active_docs}")

print(f"\n💾 HOÀN THÀNH ONE-HOT ENCODING!")
print(f"📏 Tổng categorical dimensions: {category_encoded.shape[1] + doc_type_encoded.shape[1]}")

🏷️ ONE-HOT ENCODING CHO CATEGORICAL FEATURES
📊 PHÂN TÍCH DỮ LIỆU CATEGORICAL:

📚 Các thể loại sách (Categories):
book_category
Khoa học tự nhiên      38
Công nghệ thông tin    28
Văn học                19
Triết học              13
Kinh tế                13
Giáo dục                9
Name: count, dtype: int64

📄 Các loại tài liệu (Document Types):
book_document_type
other       64
textbook    56
Name: count, dtype: int64

🔄 THỰC HIỆN ONE-HOT ENCODING:
✅ Category encoding shape: (120, 6)
📋 Category labels: ['Công nghệ thông tin', 'Giáo dục', 'Khoa học tự nhiên', 'Kinh tế', 'Triết học', 'Văn học']
✅ Document type encoding shape: (120, 2)
📋 Document type labels: ['other', 'textbook']

📋 VÍ DỤ ENCODING RESULTS:

🏷️ Category One-Hot (5 sách đầu):
   category_Công nghệ thông tin  category_Giáo dục  \
0                             0                  0   
1                             0                  1   
2                             0                  0   
3                             1   

- Title TF-IDF: 202 dimensions
- Description TF-IDF: 2000 dimensions  
- Category OneHot: 6 dimensions
- DocType OneHot: 2 dimensions
---
TỔNG: 2210 dimensions

In [11]:
# Cell 7: Kết hợp tất cả features với trọng số (Updated)
import numpy as np
from sklearn.preprocessing import normalize
from scipy import sparse

print("⚖️ KẾT HỢP TẤT CẢ FEATURES VỚI TRỌNG SỐ (UPDATED)")
print("="*50)

# Định nghĩa trọng số mới: tăng category, giảm description
weights = {
    'title': 0.25,
    'description': 0.3,    # Giảm từ 0.4 → 0.3 (-0.1)
    'category': 0.35,      # Tăng từ 0.25 → 0.35 (+0.1)
    'doc_type': 0.1
}

print("📊 TRỌNG SỐ MỚI (category cao hơn):")
for feature, weight in weights.items():
    print(f"   {feature}: {weight}")

# Kiểm tra tổng trọng số = 1
total_weight = sum(weights.values())
print(f"✅ Tổng trọng số: {total_weight}")

print(f"\n📏 KÍCH THƯỚC CÁC FEATURE MATRICES:")
print(f"   Title TF-IDF: {title_tfidf_matrix.shape}")
print(f"   Description TF-IDF: {desc_tfidf_matrix.shape}")
print(f"   Category OneHot: {category_encoded.shape}")
print(f"   DocType OneHot: {doc_type_encoded.shape}")

# 1. Chuẩn hóa từng feature matrix (L2 normalization)
print(f"\n🔄 CHUẨN HÓA CÁC FEATURE MATRICES:")

# Normalize TF-IDF matrices (đã là sparse)
title_normalized = normalize(title_tfidf_matrix, norm='l2', axis=1)
desc_normalized = normalize(desc_tfidf_matrix, norm='l2', axis=1)

# Normalize categorical matrices (dense)
category_normalized = normalize(category_encoded, norm='l2', axis=1)
doc_type_normalized = normalize(doc_type_encoded, norm='l2', axis=1)

print("✅ Đã chuẩn hóa tất cả feature matrices")

# 2. Áp dụng trọng số mới và kết hợp
print(f"\n⚖️ ÁP DỤNG TRỌNG SỐ MỚI:")

# Apply updated weights
title_weighted = title_normalized * weights['title']
desc_weighted = desc_normalized * weights['description']  # 0.3
category_weighted = category_normalized * weights['category']  # 0.35
doc_type_weighted = doc_type_normalized * weights['doc_type']

print("✅ Đã áp dụng trọng số mới cho từng feature")
print(f"   📖 Title weight: {weights['title']}")
print(f"   📄 Description weight: {weights['description']} (giảm)")
print(f"   🏷️ Category weight: {weights['category']} (tăng)")
print(f"   📋 DocType weight: {weights['doc_type']}")

# 3. Kết hợp tất cả features
print(f"\n🔗 KẾT HỢP TẤT CẢ FEATURES:")

# Convert dense arrays to sparse for efficient concatenation
category_sparse = sparse.csr_matrix(category_weighted)
doc_type_sparse = sparse.csr_matrix(doc_type_weighted)

# Concatenate all features horizontally
combined_features = sparse.hstack([
    title_weighted,      # 202 dims
    desc_weighted,       # 2000 dims
    category_sparse,     # 6 dims
    doc_type_sparse      # 2 dims
])

print(f"✅ Combined features shape: {combined_features.shape}")
print(f"📊 Tổng dimensions: {combined_features.shape[1]}")

# 4. Kiểm tra impact của trọng số mới
print(f"\n🔍 KIỂM TRA TÁC ĐỘNG TRỌNG SỐ MỚI:")

for i in range(3):
    book_title = books_df.iloc[i]['book_title']
    book_category = books_df.iloc[i]['book_category']
    feature_vector = combined_features[i].toarray().flatten()

    # Tính norm của từng component
    title_start, title_end = 0, 202
    desc_start, desc_end = 202, 2202
    cat_start, cat_end = 2202, 2208
    doc_start, doc_end = 2208, 2210

    title_norm = np.linalg.norm(feature_vector[title_start:title_end])
    desc_norm = np.linalg.norm(feature_vector[desc_start:desc_end])
    cat_norm = np.linalg.norm(feature_vector[cat_start:cat_end])
    doc_norm = np.linalg.norm(feature_vector[doc_start:doc_end])

    print(f"\n📖 Sách {i+1}: {book_title}")
    print(f"   🏷️ Category: {book_category}")
    print(f"   📖 Title component: {title_norm:.4f}")
    print(f"   📄 Description component: {desc_norm:.4f} (giảm weight)")
    print(f"   🏷️ Category component: {cat_norm:.4f} (tăng weight)")
    print(f"   📋 DocType component: {doc_norm:.4f}")
    print(f"   🎯 Total vector norm: {np.linalg.norm(feature_vector):.4f}")

# 5. Lưu thông tin với trọng số mới
feature_info = {
    'matrix': combined_features,
    'dimensions': {
        'title': (0, 202),
        'description': (202, 2202),
        'category': (2202, 2208),
        'doc_type': (2208, 2210)
    },
    'weights': weights,  # Trọng số mới
    'total_dims': combined_features.shape[1]
}

print(f"\n💾 ĐÃ HOÀN THÀNH KẾT HỢP FEATURES VỚI TRỌNG SỐ MỚI!")
print(f"🎯 Category importance tăng lên → recommendation theo chủ đề tốt hơn!")
print(f"📏 Final feature matrix: {combined_features.shape}")

⚖️ KẾT HỢP TẤT CẢ FEATURES VỚI TRỌNG SỐ (UPDATED)
📊 TRỌNG SỐ MỚI (category cao hơn):
   title: 0.25
   description: 0.3
   category: 0.35
   doc_type: 0.1
✅ Tổng trọng số: 1.0

📏 KÍCH THƯỚC CÁC FEATURE MATRICES:
   Title TF-IDF: (120, 202)
   Description TF-IDF: (120, 2000)
   Category OneHot: (120, 6)
   DocType OneHot: (120, 2)

🔄 CHUẨN HÓA CÁC FEATURE MATRICES:
✅ Đã chuẩn hóa tất cả feature matrices

⚖️ ÁP DỤNG TRỌNG SỐ MỚI:
✅ Đã áp dụng trọng số mới cho từng feature
   📖 Title weight: 0.25
   📄 Description weight: 0.3 (giảm)
   🏷️ Category weight: 0.35 (tăng)
   📋 DocType weight: 0.1

🔗 KẾT HỢP TẤT CẢ FEATURES:
✅ Combined features shape: (120, 2210)
📊 Tổng dimensions: 2210

🔍 KIỂM TRA TÁC ĐỘNG TRỌNG SỐ MỚI:

📖 Sách 1: Sáng tạo bừng cháy sức mạnh bên trong
   🏷️ Category: Triết học
   📖 Title component: 0.2500
   📄 Description component: 0.3000 (giảm weight)
   🏷️ Category component: 0.3500 (tăng weight)
   📋 DocType component: 0.1000
   🎯 Total vector norm: 0.5339

📖 Sách 2: Phong 

In [15]:
# Cell 8: Tính toán Cosine Similarity Matrix (FIXED)
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

print("📐 TÍNH TOÁN COSINE SIMILARITY MATRIX (FIXED)")
print("="*50)

# Sử dụng similarity_matrix đã tính từ cell trước
print(f"📊 SIMILARITY MATRIX INFO:")
print(f"   📏 Shape: {similarity_matrix.shape}")
print(f"   💾 Size: {similarity_matrix.nbytes / (1024*1024):.2f} MB")

# Phân tích similarity matrix (fixed)
print(f"\n📊 PHÂN TÍCH SIMILARITY MATRIX:")
print(f"   📈 Max similarity: {similarity_matrix.max():.4f}")
print(f"   📉 Min similarity: {similarity_matrix.min():.4f}")
print(f"   📊 Mean similarity: {similarity_matrix.mean():.4f}")

# Fix: Tìm top similar pairs ĐÚNG CÁCH
print(f"\n🏆 TOP 10 CẶP SÁCH TƯƠNG TỰ NHẤT (FIXED):")

# Tạo mask để loại bỏ diagonal và lower triangle
mask = np.triu(np.ones_like(similarity_matrix, dtype=bool), k=1)

# Lấy tất cả similarity scores và indices
similarities = similarity_matrix[mask]
indices = np.where(mask)

# Sort để lấy top similarities
sorted_indices = np.argsort(similarities)[::-1]  # Descending order

# Hiển thị top 10 pairs
for i in range(10):
    idx = sorted_indices[i]
    row_idx = indices[0][idx]
    col_idx = indices[1][idx]
    score = similarities[idx]

    book1 = books_df.iloc[row_idx]['book_title']
    book2 = books_df.iloc[col_idx]['book_title']
    cat1 = books_df.iloc[row_idx]['book_category']
    cat2 = books_df.iloc[col_idx]['book_category']
    doc1 = books_df.iloc[row_idx]['book_document_type']
    doc2 = books_df.iloc[col_idx]['book_document_type']

    print(f"\n🔗 Cặp {i+1} (Score: {score:.4f}):")
    print(f"   📖 Sách 1: {book1}")
    print(f"      🏷️ Category: {cat1} | 📄 Type: {doc1}")
    print(f"   📖 Sách 2: {book2}")
    print(f"      🏷️ Category: {cat2} | 📄 Type: {doc2}")

    # Kiểm tra xem có cùng category không
    same_category = "✅ Cùng category" if cat1 == cat2 else "❌ Khác category"
    print(f"   🎯 {same_category}")

# Phân tích similarity theo category
print(f"\n🏷️ PHÂN TÍCH SIMILARITY THEO CATEGORY:")
categories = books_df['book_category'].unique()

category_stats = {}
for cat in categories:
    cat_indices = books_df[books_df['book_category'] == cat].index.tolist()

    if len(cat_indices) > 1:
        # Tính similarity trong cùng category
        cat_similarities = []
        for i in range(len(cat_indices)):
            for j in range(i+1, len(cat_indices)):
                sim = similarity_matrix[cat_indices[i], cat_indices[j]]
                cat_similarities.append(sim)

        if cat_similarities:
            mean_sim = np.mean(cat_similarities)
            max_sim = np.max(cat_similarities)
            min_sim = np.min(cat_similarities)
            count = len(cat_indices)

            category_stats[cat] = {
                'mean': mean_sim,
                'max': max_sim,
                'min': min_sim,
                'count': count,
                'pairs': len(cat_similarities)
            }

            print(f"\n🏷️ {cat} ({count} sách, {len(cat_similarities)} cặp):")
            print(f"   📊 Mean similarity: {mean_sim:.4f}")
            print(f"   📈 Max similarity: {max_sim:.4f}")
            print(f"   📉 Min similarity: {min_sim:.4f}")

# Thống kê tổng quan
print(f"\n📈 THỐNG KÊ TỔNG QUAN:")
all_similarities = similarities
print(f"   📊 Tổng số cặp sách: {len(all_similarities)}")
print(f"   📈 Similarity cao nhất: {all_similarities.max():.4f}")
print(f"   📉 Similarity thấp nhất: {all_similarities.min():.4f}")
print(f"   📊 Mean similarity: {all_similarities.mean():.4f}")
print(f"   📏 Std similarity: {all_similarities.std():.4f}")

# Phân bố theo ngưỡng
high_sim = np.sum(all_similarities >= 0.7)
medium_sim = np.sum((all_similarities >= 0.4) & (all_similarities < 0.7))
low_sim = np.sum(all_similarities < 0.4)

print(f"\n📊 PHÂN BỐ SIMILARITY:")
print(f"   🔥 High (≥0.7): {high_sim} cặp ({high_sim/len(all_similarities)*100:.1f}%)")
print(f"   🔶 Medium (0.4-0.7): {medium_sim} cặp ({medium_sim/len(all_similarities)*100:.1f}%)")
print(f"   🔵 Low (<0.4): {low_sim} cặp ({low_sim/len(all_similarities)*100:.1f}%)")

print(f"\n✅ ĐÃ FIX XONG! TOP PAIRS HIỂN THỊ CHÍNH XÁC!")
print(f"🎯 SẴN SÀNG XÂY DỰNG RECOMMENDATION FUNCTION!")

📐 TÍNH TOÁN COSINE SIMILARITY MATRIX (FIXED)
📊 SIMILARITY MATRIX INFO:
   📏 Shape: (120, 120)
   💾 Size: 0.11 MB

📊 PHÂN TÍCH SIMILARITY MATRIX:
   📈 Max similarity: 1.0000
   📉 Min similarity: 0.0035
   📊 Mean similarity: 0.1582

🏆 TOP 10 CẶP SÁCH TƯƠNG TỰ NHẤT (FIXED):

🔗 Cặp 1 (Score: 0.8596):
   📖 Sách 1: Lập trình hướng đối tượng
      🏷️ Category: Công nghệ thông tin | 📄 Type: textbook
   📖 Sách 2: Lập trình hướng đối tượng Java
      🏷️ Category: Công nghệ thông tin | 📄 Type: textbook
   🎯 ✅ Cùng category

🔗 Cặp 2 (Score: 0.8057):
   📖 Sách 1: Lập trình hướng đối tượng Java
      🏷️ Category: Công nghệ thông tin | 📄 Type: textbook
   📖 Sách 2: Lập trình .Net
      🏷️ Category: Công nghệ thông tin | 📄 Type: textbook
   🎯 ✅ Cùng category

🔗 Cặp 3 (Score: 0.8003):
   📖 Sách 1: Lập trình .Net
      🏷️ Category: Công nghệ thông tin | 📄 Type: textbook
   📖 Sách 2: Lập trình Web
      🏷️ Category: Công nghệ thông tin | 📄 Type: textbook
   🎯 ✅ Cùng category

🔗 Cặp 4 (Score: 0.7999):
   

📋 TRẠNG THÁI ITEM PROFILE - ĐÃ HOÀN THÀNH:
✅ ITEM PROFILE ĐÃ XÂY DỰNG XONG:
📍 VỊ TRÍ ITEM PROFILES:

Python
# Item profiles được lưu trong:
combined_features  # Shape: (120, 2210) - ma trận feature cho 120 sách
🎯 CẤU TRÚC ITEM PROFILE CHO MỖI SÁCH:

Code
Mỗi sách có vector 2210 dimensions:
├── Title TF-IDF: [0:202] - 202 dims (weight: 0.25)
├── Description TF-IDF: [202:2202] - 2000 dims (weight: 0.3)
├── Category OneHot: [2202:2208] - 6 dims (weight: 0.35)
└── DocType OneHot: [2208:2210] - 2 dims (weight: 0.1)
📊 CHI TIẾT ITEM PROFILES:
📝 Metadata được lưu trong:

Python
feature_info = {
    'matrix': combined_features,        # Ma trận features
    'dimensions': {...},                # Mapping dimensions
    'weights': {...},                   # Trọng số features
    'total_dims': 2210                 # Tổng dimensions
}

books_df  # DataFrame chứa thông tin sách (title, category, description...)
🎯 TRẠNG THÁI HIỆN TẠI:
✅ ĐÃ HOÀN THÀNH:

✅ Text preprocessing (title, description)
✅ TF-IDF vectorization
✅ Categorical encoding (category, doc_type)
✅ Feature combination với weights
✅ Similarity matrix calculation
🚀 SẴN SÀNG CHO:

Content-Based Recommendation Function
User query processing
Real-time recommendation
Item profiles đã sẵn sàng để build recommendation system! 🎉

In [20]:
# Cell 9 Fixed: Item-Based Recommendation với MongoDB ObjectId
import pandas as pd
import numpy as np

print("🎯 XÂY DỰNG ITEM-BASED RECOMMENDATION SYSTEM (MongoDB Compatible)")
print("="*70)

def create_book_id_mapping():
    """
    Tạo mapping giữa MongoDB ObjectId và DataFrame index
    """
    # Giả sử books_df có cột 'book_id' chứa MongoDB ObjectId
    if 'book_id' not in books_df.columns:
        # Nếu chưa có, tạo fake ObjectIds để demo
        from bson import ObjectId
        books_df['book_id'] = [ObjectId() for _ in range(len(books_df))]

    # Tạo mapping dict
    objectid_to_index = {str(books_df.iloc[i]['book_id']): i for i in range(len(books_df))}
    index_to_objectid = {i: str(books_df.iloc[i]['book_id']) for i in range(len(books_df))}

    print(f"✅ Tạo mapping cho {len(books_df)} sách")
    print(f"📝 Sample mapping:")
    for i in range(min(3, len(books_df))):
        obj_id = index_to_objectid[i]
        print(f"   Index {i} ↔ ObjectId {obj_id[:12]}...")

    return objectid_to_index, index_to_objectid

def recommend_by_mongodb_id(mongodb_book_id, top_k=5, min_similarity=0.1):
    """
    Gợi ý sách tương tự dựa trên MongoDB ObjectId

    Parameters:
    - mongodb_book_id: ObjectId của sách từ MongoDB (string)
    - top_k: số lượng sách gợi ý
    - min_similarity: ngưỡng similarity tối thiểu

    Returns:
    - List of recommended books với MongoDB ObjectId
    """
    print(f"🔍 Tìm sách tương tự cho MongoDB ID: {mongodb_book_id}")

    # Convert MongoDB ObjectId to DataFrame index
    if mongodb_book_id not in objectid_to_index:
        print(f"❌ Không tìm thấy sách với ID: {mongodb_book_id}")
        return []

    df_index = objectid_to_index[mongodb_book_id]
    print(f"📍 Mapped to DataFrame index: {df_index}")

    # Sử dụng function recommendation cũ với df_index
    source_book = books_df.iloc[df_index]
    print(f"📖 Sách gốc: {source_book['book_title']}")
    print(f"🏷️ Category: {source_book['book_category']}")

    # Lấy similarity scores
    book_similarities = similarity_matrix[df_index].copy()
    book_similarities[df_index] = -1  # Loại bỏ chính nó

    # Lọc và sort
    valid_indices = np.where(book_similarities >= min_similarity)[0]
    if len(valid_indices) == 0:
        print(f"❌ Không tìm thấy sách tương tự")
        return []

    sorted_indices = valid_indices[np.argsort(book_similarities[valid_indices])[::-1]]
    top_indices = sorted_indices[:top_k]

    # Tạo kết quả với MongoDB ObjectIds
    recommendations = []

    print(f"\n🏆 TOP {len(top_indices)} SÁCH TƯƠNG TỰ:")
    print("-" * 80)

    for i, idx in enumerate(top_indices):
        book = books_df.iloc[idx]
        mongodb_id = index_to_objectid[idx]
        similarity_score = book_similarities[idx]

        rec = {
            'mongodb_book_id': mongodb_id,
            'book_title': book['book_title'],
            'book_category': book['book_category'],
            'book_document_type': book['book_document_type'],
            'similarity_score': float(similarity_score),
            'rank': i + 1
        }

        recommendations.append(rec)

        # Hiển thị
        same_category = "✅" if book['book_category'] == source_book['book_category'] else "❌"
        print(f"{i+1:2d}. 📚 {book['book_title'][:50]}")
        print(f"    🆔 MongoDB ID: {mongodb_id}")
        print(f"    🏷️ {book['book_category']} {same_category}")
        print(f"    📊 Similarity: {similarity_score:.4f}")
        print()

    return recommendations

def get_book_info_by_mongodb_id(mongodb_book_id):
    """
    Lấy thông tin sách theo MongoDB ObjectId
    """
    if mongodb_book_id not in objectid_to_index:
        return None

    df_index = objectid_to_index[mongodb_book_id]
    book = books_df.iloc[df_index]

    return {
        'mongodb_book_id': mongodb_book_id,
        'df_index': df_index,
        'book_title': book['book_title'],
        'book_category': book['book_category'],
        'book_document_type': book['book_document_type']
    }

# Tạo mapping system
objectid_to_index, index_to_objectid = create_book_id_mapping()

# Test với MongoDB ObjectId
print(f"\n🧪 TESTING VỚI MONGODB OBJECTID:")
print("=" * 50)

# Lấy một MongoDB ID để test
test_mongodb_id = index_to_objectid[0]  # Sách đầu tiên
print(f"\n🎯 Test với MongoDB ID: {test_mongodb_id}")

# Get book info
book_info = get_book_info_by_mongodb_id(test_mongodb_id)
if book_info:
    print(f"📖 Sách: {book_info['book_title']}")
    print(f"🏷️ Category: {book_info['book_category']}")

# Get recommendations
recommendations = recommend_by_mongodb_id(test_mongodb_id, top_k=3)

print(f"\n🔗 KẾT QUẢ CHO WEBSITE:")
print("=" * 30)
print("📤 API Response format:")
for rec in recommendations[:2]:  # Show first 2
    print(f"   - MongoDB ID: {rec['mongodb_book_id']}")
    print(f"     Title: {rec['book_title']}")
    print(f"     Score: {rec['similarity_score']:.4f}")

print(f"\n✅ SẴN SÀNG INTEGRATE VỚI MONGODB!")
print(f"🌐 Website có thể gọi: recommend_by_mongodb_id(bookId)")

🎯 XÂY DỰNG ITEM-BASED RECOMMENDATION SYSTEM (MongoDB Compatible)
✅ Tạo mapping cho 120 sách
📝 Sample mapping:
   Index 0 ↔ ObjectId 6887514a9e6f...
   Index 1 ↔ ObjectId 68859b92dbd8...
   Index 2 ↔ ObjectId 68886c8a4059...

🧪 TESTING VỚI MONGODB OBJECTID:

🎯 Test với MongoDB ID: 6887514a9e6faeb381cc3a43
📖 Sách: Sáng tạo bừng cháy sức mạnh bên trong
🏷️ Category: Triết học
🔍 Tìm sách tương tự cho MongoDB ID: 6887514a9e6faeb381cc3a43
📍 Mapped to DataFrame index: 0
📖 Sách gốc: Sáng tạo bừng cháy sức mạnh bên trong
🏷️ Category: Triết học

🏆 TOP 3 SÁCH TƯƠNG TỰ:
--------------------------------------------------------------------------------
 1. 📚 Suy ngẫm về thiện và ác
    🆔 MongoDB ID: 68873f11e39419950ff0afe6
    🏷️ Triết học ✅
    📊 Similarity: 0.5889

 2. 📚 Trưởng thành
    🆔 MongoDB ID: 68874eb59e6faeb381cc399a
    🏷️ Triết học ✅
    📊 Similarity: 0.5792

 3. 📚 Luận về yêu
    🆔 MongoDB ID: 6887542b9e6faeb381cc3ab6
    🏷️ Triết học ✅
    📊 Similarity: 0.5731


🔗 KẾT QUẢ CHO WEBSITE:


In [23]:
# Cell 10: Cold Start Recommendation System (Faculty-Based Mapping)
import pandas as pd
import numpy as np

print("❄️ XÂY DỰNG COLD START RECOMMENDATION SYSTEM (FACULTY MAPPING)")
print("="*70)

def get_faculty_recommendations(faculty_code, top_k=5):
    """
    Gợi ý 100% sách theo faculty code mapping

    Parameters:
    - faculty_code: mã khoa (cntt, khtn, kinh_te, giao_duc, chinh_tri, van_hoc)
    - top_k: số sách gợi ý (default: 5)

    Returns:
    - List of recommended books từ category tương ứng
    """

    # Faculty code mapping với book categories
    faculty_to_category = {
        "cntt": "Công nghệ thông tin",
        "khtn": "Khoa học tự nhiên",
        "kinh_te": "Kinh tế",
        "giao_duc": "Giáo dục",
        "chinh_tri": "Triết học",  # Map "Chính trị" → "Triết học" (available category)
        "van_hoc": "Văn học"
    }

    faculty_code_lower = faculty_code.lower().strip()
    print(f"🎓 Processing faculty code: '{faculty_code}'")

    # Kiểm tra faculty code có trong mapping không
    if faculty_code_lower not in faculty_to_category:
        print(f"❌ Faculty code '{faculty_code}' not found in mapping")
        available_codes = list(faculty_to_category.keys())
        print(f"📋 Available codes: {available_codes}")
        return []

    target_category = faculty_to_category[faculty_code_lower]
    print(f"✅ Mapped to category: '{target_category}'")

    # Kiểm tra category có tồn tại trong data không
    available_categories = books_df['book_category'].unique().tolist()
    if target_category not in available_categories:
        print(f"❌ Category '{target_category}' not found in books data")
        print(f"📋 Available categories: {available_categories}")
        return []

    # Lấy sách từ category
    category_books = books_df[books_df['book_category'] == target_category]

    if len(category_books) == 0:
        print(f"❌ No books found in category: '{target_category}'")
        return []

    print(f"📚 Found {len(category_books)} books in '{target_category}' category")

    # Lấy top K sách (theo index - có thể thay bằng rating/popularity)
    selected_books = category_books.head(top_k)

    recommendations = []
    for i, (idx, row) in enumerate(selected_books.iterrows()):
        recommendations.append({
            'mongodb_book_id': index_to_objectid[idx],
            'book_title': row['book_title'],
            'book_category': row['book_category'],
            'book_document_type': row['book_document_type'],
            'faculty_match_score': 1.0 - (i * 0.1),
            'rank': i + 1,
            'faculty_code': faculty_code_lower,
            'matched_category': target_category
        })

    print(f"✅ Generated {len(recommendations)} recommendations")
    return recommendations

def cold_start_recommendation(faculty_code=None, top_k=5):
    """
    Main cold start recommendation function

    Parameters:
    - faculty_code: mã khoa của user (None nếu không có)
    - top_k: số sách gợi ý (default: 5)

    Returns:
    - Dict with recommendations and cold_start_type for backend
    """
    result = {
        'recommendations': [],
        'cold_start_type': None,
        'message': '',
        'total_count': 0,
        'faculty_code': faculty_code,
        'matched_category': None
    }

    print(f"❄️ COLD START RECOMMENDATION")
    print(f"🎓 Faculty Code: {faculty_code}")
    print("-" * 50)

    if faculty_code and faculty_code.strip():
        # Case: User có faculty code
        faculty_recs = get_faculty_recommendations(faculty_code, top_k)

        if faculty_recs:
            result['recommendations'] = faculty_recs
            result['cold_start_type'] = 'FACULTY_BASED'
            result['message'] = f'Recommendations for faculty: {faculty_code}'
            result['total_count'] = len(faculty_recs)
            result['matched_category'] = faculty_recs[0]['matched_category']
            print(f"✅ SUCCESS: {len(faculty_recs)} faculty-based recommendations")
        else:
            # Faculty code không map được hoặc không có sách
            result['cold_start_type'] = 'SHOW_TRENDING'
            result['message'] = f'No books available for faculty: {faculty_code}'
            print(f"❌ FALLBACK: No books for faculty '{faculty_code}' → Show trending")
    else:
        # Case: User không có faculty code (guest/not logged in)
        result['cold_start_type'] = 'SHOW_TRENDING'
        result['message'] = 'No faculty information available'
        print(f"❌ FALLBACK: No faculty code → Show trending")

    return result

def display_cold_start_results(result):
    """
    Hiển thị kết quả cold start recommendations
    """
    print(f"\n📊 COLD START RESULT:")
    print(f"   🏷️ Type: {result['cold_start_type']}")
    print(f"   🎓 Faculty: {result['faculty_code']}")
    print(f"   📂 Category: {result['matched_category']}")
    print(f"   💬 Message: {result['message']}")
    print(f"   📚 Count: {result['total_count']}")

    if result['recommendations']:
        print(f"\n🏆 RECOMMENDATIONS:")
        print("-" * 70)

        for book in result['recommendations']:
            print(f"{book['rank']:2d}. 📚 {book['book_title']}")
            print(f"    🆔 {book['mongodb_book_id'][:12]}...")
            print(f"    🏷️ {book['book_category']} | 📄 {book['book_document_type']}")
            print(f"    📊 Score: {book['faculty_match_score']:.3f}")
            print()
    else:
        print(f"\n🔄 Backend should show trending/popular books")

# Test tất cả faculty codes
print(f"\n🧪 TESTING ALL FACULTY CODES:")
print("=" * 60)

faculty_codes = ["cntt", "khtn", "kinh_te", "giao_duc", "chinh_tri", "van_hoc"]

for faculty_code in faculty_codes:
    print(f"\n❄️ Test Faculty: {faculty_code.upper()}")
    result = cold_start_recommendation(faculty_code=faculty_code, top_k=5)
    display_cold_start_results(result)
    print("-" * 50)

# Test edge cases
print(f"\n🧪 TESTING EDGE CASES:")
print("=" * 40)

# Test 1: Invalid faculty code
print(f"\n❄️ Test: Invalid Faculty Code")
invalid_result = cold_start_recommendation(faculty_code="invalid_code", top_k=5)
display_cold_start_results(invalid_result)

# Test 2: No faculty code (Guest user)
print(f"\n❄️ Test: No Faculty Code (Guest)")
guest_result = cold_start_recommendation(faculty_code=None, top_k=5)
display_cold_start_results(guest_result)

# Test 3: Empty faculty code
print(f"\n❄️ Test: Empty Faculty Code")
empty_result = cold_start_recommendation(faculty_code="", top_k=5)
display_cold_start_results(empty_result)

print(f"\n✅ COLD START SYSTEM SẴN SÀNG!")
print(f"\n🌐 BACKEND INTEGRATION:")
print(f"   📋 Cold Start Types:")
print(f"      - 'FACULTY_BASED': Có recommendations → Hiển thị recommendations")
print(f"      - 'SHOW_TRENDING': Không có recommendations → Hiển thị trending books")
print(f"\n💻 API Usage:")
print(f"   // User đã đăng nhập với faculty")
print(f"   cold_start_recommendation(faculty_code='cntt', top_k=5)")
print(f"   ")
print(f"   // Guest user hoặc no faculty")
print(f"   cold_start_recommendation(faculty_code=None, top_k=5)")
print(f"\n📦 Response Format:")
print(f"   {{")
print(f"     'recommendations': [...],  // Array of books or []")
print(f"     'cold_start_type': 'FACULTY_BASED' | 'SHOW_TRENDING',")
print(f"     'message': 'Description',")
print(f"     'total_count': 5,")
print(f"     'faculty_code': 'cntt',")
print(f"     'matched_category': 'Công nghệ thông tin'")
print(f"   }}")

❄️ XÂY DỰNG COLD START RECOMMENDATION SYSTEM (FACULTY MAPPING)

🧪 TESTING ALL FACULTY CODES:

❄️ Test Faculty: CNTT
❄️ COLD START RECOMMENDATION
🎓 Faculty Code: cntt
--------------------------------------------------
🎓 Processing faculty code: 'cntt'
✅ Mapped to category: 'Công nghệ thông tin'
📚 Found 28 books in 'Công nghệ thông tin' category
✅ Generated 5 recommendations
✅ SUCCESS: 5 faculty-based recommendations

📊 COLD START RESULT:
   🏷️ Type: FACULTY_BASED
   🎓 Faculty: cntt
   📂 Category: Công nghệ thông tin
   💬 Message: Recommendations for faculty: cntt
   📚 Count: 5

🏆 RECOMMENDATIONS:
----------------------------------------------------------------------
 1. 📚 Khai thác dữ liệu với Python
    🆔 6884b3db381c...
    🏷️ Công nghệ thông tin | 📄 textbook
    📊 Score: 1.000

 2. 📚 Công nghệ J2EE
    🆔 6884bbd46ac4...
    🏷️ Công nghệ thông tin | 📄 textbook
    📊 Score: 0.900

 3. 📚 Mạng máy tính
    🆔 6884b6f5381c...
    🏷️ Công nghệ thông tin | 📄 textbook
    📊 Score: 0.800

 4. 

In [24]:
# Cell 11: Model Evaluation Framework
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

print("📊 XÂY DỰNG FRAMEWORK ĐÁNH GIÁ RECOMMENDATION SYSTEM")
print("="*60)

def evaluate_content_diversity():
    """
    Đánh giá độ đa dạng của recommendations
    """
    print("🎨 ĐÁNH GIÁ DIVERSITY:")

    # Test với 10 sách ngẫu nhiên
    sample_books = np.random.choice(len(books_df), 10, replace=False)

    diversity_scores = []
    category_coverage = []

    for book_id in sample_books:
        # Get recommendations
        similarities = similarity_matrix[book_id].copy()
        similarities[book_id] = -1
        top_5_indices = np.argsort(similarities)[-5:]

        # Tính diversity trong top 5
        top_5_books = books_df.iloc[top_5_indices]
        unique_categories = top_5_books['book_category'].nunique()
        unique_types = top_5_books['book_document_type'].nunique()

        diversity_score = (unique_categories + unique_types) / 2
        diversity_scores.append(diversity_score)
        category_coverage.append(unique_categories)

    avg_diversity = np.mean(diversity_scores)
    avg_category_coverage = np.mean(category_coverage)

    print(f"   📊 Average Diversity Score: {avg_diversity:.3f}")
    print(f"   🏷️ Average Category Coverage: {avg_category_coverage:.3f}")
    print(f"   📈 Diversity Range: {min(diversity_scores):.3f} - {max(diversity_scores):.3f}")

    return {
        'avg_diversity': avg_diversity,
        'avg_category_coverage': avg_category_coverage,
        'diversity_scores': diversity_scores
    }

def evaluate_category_precision():
    """
    Đánh giá precision trong cùng category
    """
    print("\n🎯 ĐÁNH GIÁ CATEGORY PRECISION:")

    category_precisions = {}

    for category in books_df['book_category'].unique():
        category_books = books_df[books_df['book_category'] == category].index.tolist()

        if len(category_books) < 2:
            continue

        precisions = []

        for book_id in category_books[:5]:  # Test 5 sách đầu mỗi category
            similarities = similarity_matrix[book_id].copy()
            similarities[book_id] = -1
            top_5_indices = np.argsort(similarities)[-5:]

            # Đếm số sách cùng category trong top 5
            same_category_count = sum(1 for idx in top_5_indices
                                    if books_df.iloc[idx]['book_category'] == category)

            precision = same_category_count / 5
            precisions.append(precision)

        category_precisions[category] = np.mean(precisions)
        print(f"   📚 {category}: {np.mean(precisions):.3f}")

    overall_precision = np.mean(list(category_precisions.values()))
    print(f"   🏆 Overall Category Precision: {overall_precision:.3f}")

    return category_precisions

def evaluate_similarity_distribution():
    """
    Phân tích phân phối similarity scores
    """
    print("\n📈 PHÂN TÍCH SIMILARITY DISTRIBUTION:")

    # Lấy tất cả similarity scores (loại bỏ diagonal)
    upper_triangle = np.triu(similarity_matrix, k=1)
    non_zero_similarities = upper_triangle[upper_triangle > 0]

    print(f"   📊 Total similarity pairs: {len(non_zero_similarities)}")
    print(f"   📈 Mean similarity: {np.mean(non_zero_similarities):.4f}")
    print(f"   📊 Std similarity: {np.std(non_zero_similarities):.4f}")
    print(f"   🔼 Max similarity: {np.max(non_zero_similarities):.4f}")
    print(f"   🔽 Min similarity: {np.min(non_zero_similarities):.4f}")

    # Phân tích percentiles
    percentiles = [25, 50, 75, 90, 95, 99]
    print(f"   📊 Percentiles:")
    for p in percentiles:
        val = np.percentile(non_zero_similarities, p)
        print(f"      {p}th: {val:.4f}")

    return {
        'mean': np.mean(non_zero_similarities),
        'std': np.std(non_zero_similarities),
        'max': np.max(non_zero_similarities),
        'min': np.min(non_zero_similarities),
        'percentiles': {p: np.percentile(non_zero_similarities, p) for p in percentiles}
    }

def evaluate_cold_start_coverage():
    """
    Đánh giá coverage của cold start system
    """
    print("\n❄️ ĐÁNH GIÁ COLD START COVERAGE:")

    faculty_codes = ["cntt", "khtn", "kinh_te", "giao_duc", "chinh_tri", "van_hoc"]
    coverage_results = {}

    for faculty_code in faculty_codes:
        result = cold_start_recommendation(faculty_code=faculty_code, top_k=5)

        coverage_results[faculty_code] = {
            'has_recommendations': result['cold_start_type'] == 'FACULTY_BASED',
            'count': result['total_count'],
            'category': result.get('matched_category', None)
        }

        status = "✅" if result['cold_start_type'] == 'FACULTY_BASED' else "❌"
        print(f"   {status} {faculty_code}: {result['total_count']} books")

    success_rate = sum(1 for r in coverage_results.values() if r['has_recommendations']) / len(faculty_codes)
    print(f"   🏆 Faculty Coverage Rate: {success_rate:.1%}")

    return coverage_results

def comprehensive_evaluation():
    """
    Đánh giá toàn diện system
    """
    print("🔍 COMPREHENSIVE SYSTEM EVALUATION")
    print("=" * 50)

    results = {}

    # 1. Diversity evaluation
    results['diversity'] = evaluate_content_diversity()

    # 2. Category precision
    results['category_precision'] = evaluate_category_precision()

    # 3. Similarity distribution
    results['similarity_dist'] = evaluate_similarity_distribution()

    # 4. Cold start coverage
    results['cold_start'] = evaluate_cold_start_coverage()

    return results

# Chạy đánh giá toàn diện
print("\n🚀 BẮT ĐẦU ĐÁNH GIÁ SYSTEM...")
evaluation_results = comprehensive_evaluation()

print(f"\n📊 TỔNG KẾT ĐÁNH GIÁ:")
print("=" * 40)
print(f"✅ Content Diversity: {evaluation_results['diversity']['avg_diversity']:.3f}")
print(f"✅ Category Precision: {np.mean(list(evaluation_results['category_precision'].values())):.3f}")
print(f"✅ Mean Similarity: {evaluation_results['similarity_dist']['mean']:.4f}")
print(f"✅ Cold Start Coverage: {sum(1 for r in evaluation_results['cold_start'].values() if r['has_recommendations'])/6:.1%}")

📊 XÂY DỰNG FRAMEWORK ĐÁNH GIÁ RECOMMENDATION SYSTEM

🚀 BẮT ĐẦU ĐÁNH GIÁ SYSTEM...
🔍 COMPREHENSIVE SYSTEM EVALUATION
🎨 ĐÁNH GIÁ DIVERSITY:
   📊 Average Diversity Score: 1.050
   🏷️ Average Category Coverage: 1.000
   📈 Diversity Range: 1.000 - 1.500

🎯 ĐÁNH GIÁ CATEGORY PRECISION:
   📚 Triết học: 1.000
   📚 Giáo dục: 1.000
   📚 Kinh tế: 1.000
   📚 Công nghệ thông tin: 1.000
   📚 Văn học: 1.000
   📚 Khoa học tự nhiên: 1.000
   🏆 Overall Category Precision: 1.000

📈 PHÂN TÍCH SIMILARITY DISTRIBUTION:
   📊 Total similarity pairs: 7140
   📈 Mean similarity: 0.1512
   📊 Std similarity: 0.1976
   🔼 Max similarity: 0.8596
   🔽 Min similarity: 0.0035
   📊 Percentiles:
      25th: 0.0295
      50th: 0.0609
      75th: 0.1100
      90th: 0.5252
      95th: 0.5614
      99th: 0.6449

❄️ ĐÁNH GIÁ COLD START COVERAGE:
❄️ COLD START RECOMMENDATION
🎓 Faculty Code: cntt
--------------------------------------------------
🎓 Processing faculty code: 'cntt'
✅ Mapped to category: 'Công nghệ thông tin'
📚 Fou

In [26]:
# Simple Test Cases using existing functions
print("🧪 SIMPLE USER TEST CASES")
print(f"📅 Test Date: 2025-08-04 05:01:54 UTC")
print(f"👤 Test Admin: Tuan9112")
print("="*50)

# Test 1: Guest User (No faculty, no history)
print("\n👤 TEST 1: GUEST USER")
print("-" * 30)
guest_result = cold_start_recommendation(faculty_code=None, top_k=5)
print(f"✅ Type: {guest_result['cold_start_type']}")
print(f"📚 Count: {guest_result['total_count']}")
print(f"💭 Message: {guest_result['message']}")

# Test 2: New IT Student (Has faculty, no history)
print("\n👤 TEST 2: NEW IT STUDENT")
print("-" * 30)
it_result = cold_start_recommendation(faculty_code='cntt', top_k=5)
print(f"✅ Type: {it_result['cold_start_type']}")
print(f"📚 Count: {it_result['total_count']}")
if it_result['recommendations']:
    print(f"📖 Sample: {it_result['recommendations'][0]['book_title'][:40]}...")

# Test 3: Economics Student
print("\n👤 TEST 3: ECONOMICS STUDENT")
print("-" * 30)
econ_result = cold_start_recommendation(faculty_code='kinh_te', top_k=5)
print(f"✅ Type: {econ_result['cold_start_type']}")
print(f"📚 Count: {econ_result['total_count']}")

# Test 4: User with History (viewed an IT book)
print("\n👤 TEST 4: USER WITH HISTORY")
print("-" * 30)
# User đã xem sách "Lập trình song song" (index 19)
viewed_book_id = index_to_objectid[19]
book_info = books_df.iloc[19]
print(f"📖 Viewed: {book_info['book_title']}")
print(f"🏷️ Category: {book_info['book_category']}")

content_recs = recommend_by_mongodb_id(viewed_book_id, top_k=5)
print(f"✅ Generated: {len(content_recs)} recommendations")
if content_recs:
    print(f"📚 Top rec: {content_recs[0]['book_title'][:40]}...")
    print(f"📊 Similarity: {content_recs[0]['similarity_score']:.3f}")

# Test 5: User with Cross-domain Interest (viewed Literature book)
print("\n👤 TEST 5: CROSS-DOMAIN USER")
print("-" * 30)
# User xem sách văn học
lit_book_id = index_to_objectid[67]  # Assuming index 67 is literature
lit_book_info = books_df.iloc[67]
print(f"📖 Viewed: {lit_book_info['book_title']}")
print(f"🏷️ Category: {lit_book_info['book_category']}")

lit_recs = recommend_by_mongodb_id(lit_book_id, top_k=5)
print(f"✅ Generated: {len(lit_recs)} recommendations")

# Test 6: Invalid Faculty Code
print("\n👤 TEST 6: INVALID FACULTY")
print("-" * 30)
invalid_result = cold_start_recommendation(faculty_code='y_khoa', top_k=5)
print(f"✅ Type: {invalid_result['cold_start_type']}")
print(f"💭 Message: {invalid_result['message']}")

# Summary
print("\n📊 TEST SUMMARY:")
print("="*25)
print("✅ Guest User: Show trending (backend handles)")
print("✅ New IT Student: 5 CNTT books")
print("✅ Economics Student: 5 Kinh tế books")
print("✅ User with History: Content-based recommendations")
print("✅ Cross-domain: Literature recommendations")
print("✅ Invalid Faculty: Fallback to trending")

print(f"\n🎯 ALL USER SCENARIOS COVERED!")
print(f"🚀 System ready for production!")

🧪 SIMPLE USER TEST CASES
📅 Test Date: 2025-08-04 05:01:54 UTC
👤 Test Admin: Tuan9112

👤 TEST 1: GUEST USER
------------------------------
❄️ COLD START RECOMMENDATION
🎓 Faculty Code: None
--------------------------------------------------
❌ FALLBACK: No faculty code → Show trending
✅ Type: SHOW_TRENDING
📚 Count: 0
💭 Message: No faculty information available

👤 TEST 2: NEW IT STUDENT
------------------------------
❄️ COLD START RECOMMENDATION
🎓 Faculty Code: cntt
--------------------------------------------------
🎓 Processing faculty code: 'cntt'
✅ Mapped to category: 'Công nghệ thông tin'
📚 Found 28 books in 'Công nghệ thông tin' category
✅ Generated 5 recommendations
✅ SUCCESS: 5 faculty-based recommendations
✅ Type: FACULTY_BASED
📚 Count: 5
📖 Sample: Khai thác dữ liệu với Python...

👤 TEST 3: ECONOMICS STUDENT
------------------------------
❄️ COLD START RECOMMENDATION
🎓 Faculty Code: kinh_te
--------------------------------------------------
🎓 Processing faculty code: 'kinh_te'
✅ Ma

In [31]:
# Cell 12: Save Model to /content/drive/MyDrive/Ebooks - FIXED VERSION
import pickle
import json
import pandas as pd
import numpy as np
import os

print("💾 SAVING MODEL TO /content/drive/MyDrive/Ebooks")
print(f"📅 Current Date and Time: 2025-08-04 05:19:28 UTC")
print(f"👤 Current User's Login: Tuan9112")
print("="*65)

# Đường dẫn chính xác cho Google Colab
model_save_path = "/content/drive/MyDrive/Ebooks"

# Kiểm tra và tạo thư mục nếu cần
print(f"📍 Target Directory: {model_save_path}")
if not os.path.exists(model_save_path):
    print("🔧 Creating directory...")
    os.makedirs(model_save_path, exist_ok=True)

print(f"✅ Directory exists: {os.path.exists(model_save_path)}")
print(f"✅ Directory writable: {os.access(model_save_path, os.W_OK)}")

def save_model_to_google_drive():
    """
    Lưu model vào Google Drive Ebooks folder
    """
    print(f"\n💾 SAVING MODEL FILES TO GOOGLE DRIVE...")

    saved_files = []

    try:
        # 1. Save similarity matrix
        similarity_file = os.path.join(model_save_path, 'similarity_matrix.pkl')
        with open(similarity_file, 'wb') as f:
            pickle.dump(similarity_matrix, f)
        saved_files.append('similarity_matrix.pkl')
        print(f"   ✅ {similarity_file}")

        # 2. Save index to ObjectId mapping
        index_mapping_file = os.path.join(model_save_path, 'index_to_objectid.json')
        json_mapping = {str(k): str(v) for k, v in index_to_objectid.items()}
        with open(index_mapping_file, 'w') as f:
            json.dump(json_mapping, f, indent=2)
        saved_files.append('index_to_objectid.json')
        print(f"   ✅ {index_mapping_file}")

        # 3. Save ObjectId to index mapping (reverse)
        objectid_mapping_file = os.path.join(model_save_path, 'objectid_to_index.json')
        objectid_to_index = {str(v): k for k, v in index_to_objectid.items()}
        with open(objectid_mapping_file, 'w') as f:
            json.dump(objectid_to_index, f, indent=2)
        saved_files.append('objectid_to_index.json')
        print(f"   ✅ {objectid_mapping_file}")

        # 4. Save books dataframe
        books_pkl_file = os.path.join(model_save_path, 'books_model_data.pkl')
        books_df.to_pickle(books_pkl_file)
        saved_files.append('books_model_data.pkl')
        print(f"   ✅ {books_pkl_file}")

        # 5. Save faculty mapping
        faculty_file = os.path.join(model_save_path, 'faculty_mapping.json')
        faculty_mapping = {
            "cntt": "Công nghệ thông tin",
            "khtn": "Khoa học tự nhiên",
            "kinh_te": "Kinh tế",
            "giao_duc": "Giáo dục",
            "chinh_tri": "Triết học",
            "van_hoc": "Văn học"
        }

        with open(faculty_file, 'w', encoding='utf-8') as f:
            json.dump(faculty_mapping, f, ensure_ascii=False, indent=2)
        saved_files.append('faculty_mapping.json')
        print(f"   ✅ {faculty_file}")

        # 6. Save model metadata
        metadata_file = os.path.join(model_save_path, 'model_metadata.json')
        metadata = {
            "model_info": {
                "type": "content_based_recommendation",
                "version": "1.0.0",
                "created_date": "2025-08-04 05:19:28",
                "created_by": "Tuan9112",
                "location": "/content/drive/MyDrive/Ebooks"
            },
            "data_stats": {
                "total_books": len(books_df),
                "similarity_matrix_shape": list(similarity_matrix.shape),
                "categories": books_df['book_category'].unique().tolist(),
                "document_types": books_df['book_document_type'].unique().tolist()
            },
            "technical_details": {
                "feature_engineering": "TF-IDF on combined features",
                "similarity_metric": "cosine_similarity",
                "cold_start_strategy": "Faculty-based filtering"
            },
            "performance": {
                "content_diversity": 1.050,
                "category_precision": 1.000,
                "mean_similarity": 0.1512
            }
        }

        with open(metadata_file, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        saved_files.append('model_metadata.json')
        print(f"   ✅ {metadata_file}")

        return saved_files

    except Exception as e:
        print(f"   ❌ Error saving files: {str(e)}")
        return []

def create_deployment_guide():
    """
    Tạo deployment guide riêng biệt
    """
    print("📖 Creating deployment guide...")

    guide_content = [
        "# Book Recommendation Model - Deployment Guide",
        "",
        "## Model Information",
        "- Created: 2025-08-04 05:19:28 UTC",
        "- Created by: Tuan9112",
        "- Type: Content-based Recommendation System",
        "- Version: 1.0.0",
        "",
        "## Production Files",
        "1. similarity_matrix.pkl - Main recommendation engine",
        "2. index_to_objectid.json - Maps index to MongoDB ObjectId",
        "3. objectid_to_index.json - Maps ObjectId to index",
        "4. books_model_data.pkl - Books dataset",
        "5. faculty_mapping.json - Faculty mappings",
        "6. model_metadata.json - Model information",
        "",
        "## Usage Example",
        "```python",
        "import pickle",
        "import json",
        "",
        "# Load model",
        "with open('similarity_matrix.pkl', 'rb') as f:",
        "    similarity_matrix = pickle.load(f)",
        "",
        "# Load mappings",
        "with open('objectid_to_index.json', 'r') as f:",
        "    objectid_to_index = json.load(f)",
        "```",
        "",
        "## Performance",
        "- Content Diversity: 1.050",
        "- Category Precision: 1.000",
        "- Cold Start Coverage: 100%",
        "",
        "Ready for production deployment!"
    ]

    guide_file = os.path.join(model_save_path, 'DEPLOYMENT_GUIDE.txt')
    with open(guide_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(guide_content))

    print(f"   ✅ {guide_file}")
    return 'DEPLOYMENT_GUIDE.txt'

def verify_saved_files():
    """
    Verify files đã lưu
    """
    print(f"\n🔍 VERIFYING SAVED FILES...")

    expected_files = [
        'similarity_matrix.pkl',
        'index_to_objectid.json',
        'objectid_to_index.json',
        'books_model_data.pkl',
        'faculty_mapping.json',
        'model_metadata.json',
        'DEPLOYMENT_GUIDE.txt'
    ]

    verified_count = 0
    total_size = 0

    for file in expected_files:
        file_path = os.path.join(model_save_path, file)
        if os.path.exists(file_path):
            size = os.path.getsize(file_path)
            total_size += size
            verified_count += 1
            print(f"   ✅ {file} ({size:,} bytes)")
        else:
            print(f"   ❌ {file} - NOT FOUND")

    print(f"\n📊 SUMMARY:")
    print(f"   Files verified: {verified_count}/{len(expected_files)}")
    print(f"   Total size: {total_size:,} bytes ({total_size/1024/1024:.2f} MB)")

    return verified_count == len(expected_files)

def test_model_loading():
    """
    Test loading model từ saved files
    """
    print(f"\n🧪 TESTING MODEL LOADING...")

    try:
        # Test similarity matrix
        similarity_file = os.path.join(model_save_path, 'similarity_matrix.pkl')
        with open(similarity_file, 'rb') as f:
            test_similarity = pickle.load(f)
        print(f"   ✅ Similarity matrix: {test_similarity.shape}")

        # Test mappings
        mapping_file = os.path.join(model_save_path, 'index_to_objectid.json')
        with open(mapping_file, 'r') as f:
            test_mapping = json.load(f)
        print(f"   ✅ Mapping: {len(test_mapping)} entries")

        # Test functionality
        test_similarities = test_similarity[0]
        top_similar = sorted(range(len(test_similarities)),
                           key=lambda i: test_similarities[i], reverse=True)[1:6]
        print(f"   ✅ Recommendations: {len(top_similar)} generated")

        return True

    except Exception as e:
        print(f"   ❌ Test failed: {str(e)}")
        return False

# Execute save process
try:
    print("🚀 STARTING SAVE PROCESS...")

    # Save main model files
    saved_files = save_model_to_google_drive()

    if saved_files:
        print(f"\n✅ MAIN FILES SAVED!")

        # Create deployment guide
        guide_file = create_deployment_guide()
        saved_files.append(guide_file)

        # Verify all files
        verification_success = verify_saved_files()

        if verification_success:
            print(f"\n🔍 VERIFICATION: ✅ PASSED")

            # Test loading
            test_success = test_model_loading()

            if test_success:
                print(f"\n🧪 FUNCTIONALITY TEST: ✅ PASSED")
                print(f"\n🎉 MODEL DEPLOYMENT COMPLETE!")
                print(f"\n📍 LOCATION: {model_save_path}")
                print(f"📁 FILES READY: {len(saved_files)}")
                print(f"\n🚀 READY FOR API DEVELOPMENT!")
            else:
                print(f"\n❌ FUNCTIONALITY TEST FAILED")
        else:
            print(f"\n❌ VERIFICATION FAILED")
    else:
        print(f"\n❌ SAVE FAILED")

except Exception as e:
    print(f"\n🚨 ERROR: {str(e)}")
    print("🔧 Please check variables and try again")

print(f"\n📍 MODEL SAVED TO: /content/drive/MyDrive/Ebooks")

💾 SAVING MODEL TO /content/drive/MyDrive/Ebooks
📅 Current Date and Time: 2025-08-04 05:19:28 UTC
👤 Current User's Login: Tuan9112
📍 Target Directory: /content/drive/MyDrive/Ebooks
✅ Directory exists: True
✅ Directory writable: True
🚀 STARTING SAVE PROCESS...

💾 SAVING MODEL FILES TO GOOGLE DRIVE...
   ✅ /content/drive/MyDrive/Ebooks/similarity_matrix.pkl
   ✅ /content/drive/MyDrive/Ebooks/index_to_objectid.json
   ✅ /content/drive/MyDrive/Ebooks/objectid_to_index.json
   ✅ /content/drive/MyDrive/Ebooks/books_model_data.pkl
   ✅ /content/drive/MyDrive/Ebooks/faculty_mapping.json
   ✅ /content/drive/MyDrive/Ebooks/model_metadata.json

✅ MAIN FILES SAVED!
📖 Creating deployment guide...
   ✅ /content/drive/MyDrive/Ebooks/DEPLOYMENT_GUIDE.txt

🔍 VERIFYING SAVED FILES...
   ✅ similarity_matrix.pkl (115,362 bytes)
   ✅ index_to_objectid.json (4,332 bytes)
   ✅ objectid_to_index.json (4,092 bytes)
   ✅ books_model_data.pkl (762,712 bytes)
   ✅ faculty_mapping.json (189 bytes)
   ✅ model_metada

In [32]:
print("\n🎯 RANKING CÁC HÀM THEO MỨC ĐỘ QUAN TRỌNG")
print("="*50)

critical_functions = [
    ("⭐⭐⭐ CRITICAL", "create_combined_features()", "Tạo features từ text"),
    ("⭐⭐⭐ CRITICAL", "cosine_similarity()", "Tính độ tương đồng"),
    ("⭐⭐⭐ CRITICAL", "get_recommendations_by_objectid()", "Engine chính"),
    ("⭐⭐ IMPORTANT", "TfidfVectorizer", "Chuyển text→numbers"),
    ("⭐⭐ IMPORTANT", "index_to_objectid mapping", "Kết nối với database"),
    ("⭐ HELPFUL", "get_recommendations_by_faculty()", "Cold start"),
    ("⭐ HELPFUL", "evaluate_model()", "Đánh giá chất lượng")
]

for importance, function, description in critical_functions:
    print(f"   {importance}: {function}")
    print(f"      → {description}")


🎯 RANKING CÁC HÀM THEO MỨC ĐỘ QUAN TRỌNG
   ⭐⭐⭐ CRITICAL: create_combined_features()
      → Tạo features từ text
   ⭐⭐⭐ CRITICAL: cosine_similarity()
      → Tính độ tương đồng
   ⭐⭐⭐ CRITICAL: get_recommendations_by_objectid()
      → Engine chính
   ⭐⭐ IMPORTANT: TfidfVectorizer
      → Chuyển text→numbers
   ⭐⭐ IMPORTANT: index_to_objectid mapping
      → Kết nối với database
   ⭐ HELPFUL: get_recommendations_by_faculty()
      → Cold start
   ⭐ HELPFUL: evaluate_model()
      → Đánh giá chất lượng
